In [11]:
# Carga de modulos a emplear
import sys
import os
import pandas as pd

# Agregar la carpeta raíz del proyecto a las rutas de búsqueda de Python
base_dir = os.getcwd()  # Obtiene el directorio actual (donde está la libreta)
sys.path.append(base_dir)  # Agrega la carpeta raíz del proyecto a las rutas de búsqueda

# Importar las funciones desde helpers.importar_data
from helpers.importar_data import cargar_datos_excel

# Extracción de datos

In [3]:
# Definimos las pestañas a cargar
sheets_to_load = ["Base Ventas", "Productos", "Ciudad"]

# Cargamos los datos desde el Excel
datos = cargar_datos_excel(base_dir, sheets=sheets_to_load)

# Accesamos a los Dataframes
ventas_df = datos["Base Ventas"]
productos_df = datos["Productos"]
ciudad_df = datos["Ciudad"]

# Limpieza del dato

## Base Ventas
**Cod_orden** tiene la pinta de ser un identificador unico (**Llave primaria**) para cada pedido, por lo cual verificariamos que no existan duplicados ni valores nulos.

In [6]:
print(f"Duplicados en Cod_orden: {ventas_df['Cod_orden'].duplicated().sum()}")
print(f"Valores nulos en Cod_orden: {ventas_df['Cod_orden'].isnull().sum()}")

Duplicados en Cod_orden: 0
Valores nulos en Cod_orden: 0


**Cod_Producto** tiene los codigos numericos del codigo de cada producto, en este caso seria una **llave foranea** de la **tabla Productos** en la columna que lleva el mismo nombre **Cod_Producto**.

Vamos a confirmar que todos los valores correspondan a codigos validos y consistentes y vamos a verificar esta columna con la tabla Productos para verificar la integridad (relación con llaves foráneas).

In [8]:
print(f"Valores únicos en Cod_Producto: {ventas_df['Cod_Producto'].nunique()}")

Valores únicos en Cod_Producto: 12


Tenemos 12 valores unicos de productos en la columna Cod_Producto, lo que es consistente con los 12 diferentes productos de la tabla Productos.

**Cod_Ciudad** tiene los codigos numericos de diferentes ciudades, en este caso seria una **llave foranea** de la **tabla Ciudad** en la columna con el mismo nombre *Cod_Ciudad*.

Vamos a segurarnos que todos los codigos correspondan a valores válidos de la tabla Ciudad y verificar si hay valores nulos.

In [9]:
print(f"Valores únicos en Cod_Ciudad: {ventas_df['Cod_Ciudad'].nunique()}")

Valores únicos en Cod_Ciudad: 5


Tenemos 5 valores unicos de productos en la columna Cod_Ciudad, lo que es consistente con los 5 diferentes productos de la tabla Ciudad.

**Unidades_Vendidas** contiene los valores numericos de ventas de los productos en una fecha determinada. Vamos a verificar si existen valores negativos o cero, asi como revisar la distribucion de las ventas para detectar valores atipicos.

In [10]:
print(ventas_df['Unidades_Vendidas'].describe())
print(f"Valores negativos en Unidades_Vendidas: {(ventas_df['Unidades_Vendidas'] < 0).sum()}")

count    1299.000000
mean        3.099307
std         1.424037
min         1.000000
25%         2.000000
50%         3.000000
75%         4.000000
max         5.000000
Name: Unidades_Vendidas, dtype: float64
Valores negativos en Unidades_Vendidas: 0


La desviación estándar es 1.42, lo que indica que las ventas están moderadamente dispersas alrededor de la media.

Dado que la media y la mediana son casi iguales, la distribución parece ser simétrica, sin una presencia significativa de valores extremos (outliers).

**Fecha_Orden** contiene las fechas de las unidades vendidas, vamos a confirmar que los valores esten en formato de fecha y buscaremos valores fuera del rango esperado (fechas futuras o inconsistencias).

In [ ]:
# Cambiamos el tipo de dato a datetime.
ventas_df['Fecha_orden'] = pd.to_datetime(ventas_df['Fecha_orden'], errors='coerce')
print(f"Fechas inválidas: {ventas_df['Fecha_orden'].isnull().sum()}")
print(f"Rango de fechas: {ventas_df['Fecha_orden'].min()} - {ventas_df['Fecha_orden'].max()}")

Fechas inválidas: 0
Rango de fechas: 2024-01-01 00:00:00 - 2024-03-31 00:00:00


No tenemos formatos de fecha invalidas y el rango de fechas va del 01 de enero del 2024 al 31 de marzo del 2024.

**Tipo_Venta** contiene el giro de quien realizo la venta, Nos vamos a segurar que todos los valores sean consistentes y buscar valores inesperados o mal escritos.

In [13]:
print(f"Valores únicos en Tipo_Venta: {ventas_df['Tipo_Venta'].unique()}")

Valores únicos en Tipo_Venta: ['TIENDA' 'DOMICILIO']


Solo existen dos valores TIENDA y DOMICILIO.

## Productos

**Cod_Producto** contiene el codigo para cada producto, es la **llave primaria de esta tabla** y es una **llave foranea en la tabla Base Ventas**. 

Vamos a verificar que no haya duplicados y asegurarnos que no existan valores nulos.

In [14]:
print(f"Duplicados en Cod_Producto: {productos_df['Cod_Producto'].duplicated().sum()}")
print(f"Valores nulos en Cod_Producto: {productos_df['Cod_Producto'].isnull().sum()}")

Duplicados en Cod_Producto: 0
Valores nulos en Cod_Producto: 0


Nom_producto contiene los nombres de los productos, vamos a revisar si tenemos valores duplicados y comprobar que no existan nombres en blanco o nulos.

In [15]:
print(f"Duplicados en Nom_producto: {productos_df['Nom_producto'].duplicated().sum()}")
print(f"Valores nulos en Nom_producto: {productos_df['Nom_producto'].isnull().sum()}")

Duplicados en Nom_producto: 0
Valores nulos en Nom_producto: 0


Precio_Unitario y Costo_Unitario contienen los costos y precios unitarios de los productos. Vamos a convertir los valores a formato numerico para conservar solo numeros (la columna cuanta con simbolos como *$* y comas). 

Vamos a calcular estadisticas descriptivas para ambas columnas.

Nos vamos a asegurar de que Costo_Unitario sea menor o igual a Precio_Unitario, pues esta no deberian haber productos que se vendan por debajo de su costo.

Revisaremos posibles valores extremos o atípicos.